<a href="https://colab.research.google.com/github/quinzit/Master-project/blob/main/test%201_PDF_to_dataset_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pdfplumber


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 81.1 MB/s eta 0:00:00


In [24]:
import logging
logging.getLogger("pdfminer").setLevel(logging.ERROR)


In [25]:
import pdfplumber
import pandas as pd
import os

In [41]:
from google.colab import files
uploaded = files.upload()


Saving Test1.zip to Test1.zip


In [42]:
import zipfile
import os

# Extract uploaded ZIP file
zip_path = "Test1.zip"  # make sure this matches the file you upload
extract_dir = "pdfs"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("✅ Files extracted to:", extract_dir)


✅ Files extracted to: pdfs


In [43]:
import os

print("📁 Files inside extracted folder:")
for root, dirs, files in os.walk("pdfs"):
    for file in files:
        print(os.path.join(root, file))


📁 Files inside extracted folder:
pdfs/Test/order-O2025-1799.pdf
pdfs/Test/order-O2025-3238.pdf
pdfs/Test/order-O2025-1785.pdf
pdfs/Test/order-O2025-3248.pdf
pdfs/Test/order-O2025-1782.pdf
pdfs/Test/order-O2025-1789.pdf
pdfs/Test/order-O2025-1796.pdf
pdfs/Test/order-O2025-1795.pdf
pdfs/Test/order-O2025-1798.pdf
pdfs/Test/order-O2025-3244.pdf
pdfs/Test/order-O2025-1787.pdf
pdfs/Test/order-O2025-1794.pdf
pdfs/Test/order-O2025-1781.pdf
pdfs/Test/order-O2025-1786.pdf


In [47]:
import pdfplumber
import pandas as pd
import re

pdf_dir = "pdfs/Test"
all_data = [] # Will store all extracted rows

# Loop through each PDF file
for filename in os.listdir(pdf_dir):
    if filename.endswith(".pdf"):
        full_path = os.path.join(pdf_dir, filename)
        # Open the first page of each PDF
        with pdfplumber.open(full_path) as pdf:
            page = pdf.pages[0]
            lines = page.extract_text().split("\n")

            # Extract the Date (format: dd-mm-yyyy)
            extracted_date = None
            for line in lines:
                if "Date" in line:
                    parts = line.split()
                    for part in parts:
                        if re.match(r"\d{2}-\d{2}-\d{4}", part):
                            extracted_date = part
                            break

            # Find product rows
            table_started = False
            rows = []
            for line in lines:
               # Header line that indicates start of the table
                if line.strip().startswith("Product code"):
                    table_started = True
                    continue
                if table_started:
                  # Stop at the "Total" line
                    if line.strip().startswith("Total"):
                        break
                    rows.append(line.strip())

            # Process each product row
            for row in rows:
                try:
                    # Remove price and total by splitting on the last "€"
                    parts = row.rsplit("€", 2)[0].strip().rsplit(" ", 1)
                    rest = parts[0].strip()
                    amount = int(parts[1].strip())

                    # Match product code formats:
                    # 1508A / 12-2025, 8931 / 03-2025, or 3685 -> sample the item code
                    match = re.match(r"^([A-Z0-9]+(?:\s*/\s*\d{2,4}-\d{4})?)\s+(.*)", rest) # the first index is regex pattern, the second index is input text: everything before the amount (price info removed)
                    if match:
                      # Clean and extract product code and name
                        product_code = match.group(1).replace(" ", "")
                        name = match.group(2).strip()
                    else:
                      # Fallback if pattern doesn't match
                        tokens = rest.split(" ", 1)
                        product_code = tokens[0].strip()
                        name = tokens[1].strip() if len(tokens) > 1 else ""
                      # Save to result list
                    all_data.append({
                        "Product code": product_code,
                        "Name": name,
                        "Amount": amount,
                        "Date": extracted_date,
                        "File": filename
                    })
                except Exception as e:
                  # Skip row if there was an issue with parsing
                    print(f"⚠️ Skipped row due to error: {row}\n➡️ Reason: {e}")

In [48]:
#export to csv file:
if all_data:
    final_df = pd.DataFrame(all_data)                                           # Convert list of dictionaries to DataFrame
    output_file = "extracted_data.xlsx"                                         # Set Excel output filename
    final_df.to_excel(output_file, index=False)                                 # Save to Excel using openpyxl engine

# Trigger download in browser
    from google.colab import files
    files.download(output_file)
    print("✅ Done! Excel file is ready to download.")
else:
    print("❌ No data extracted. Please check PDF content.")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Done! Excel file is ready to download.
